                            Project Milestone 3 

Model Building & Evaluation steps:

1. The objective of the project is whether a loan provided will likely be charged off or not. This is a classification problem and hence a classification model is used.

2. The most impactful features determined in project milestone 2 are populated as X- the predictor variables and loan_status_charged off is assigned as target variable.

3. Using sklearn's traintestsplit, the dataset is split into Xtrain, Xtest, ytrain and ytest. 

4. GridSearchCv is used to determine the model that will provide more accurate results among - Logistic Regression and Random Forest.

5. Model accuracy score is determined on x_test, y_test dataset.

In [1]:
# Load required libraries

import os
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns

In [2]:
# Load the data into dataframe and review the data

loanapp_df = pd.read_csv("/Users/manikantanvenkataraman/Downloads/archive (1)/loan/loan.csv")
loanapp_df.head()

/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (19,55) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m
0,1077501,1296599,5000.0,5000.0,4975.0,36 months,10.65,162.87,B,B2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1077430,1314167,2500.0,2500.0,2500.0,60 months,15.27,59.83,C,C4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1077175,1313524,2400.0,2400.0,2400.0,36 months,15.96,84.33,C,C5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1076863,1277178,10000.0,10000.0,10000.0,36 months,13.49,339.31,C,C1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1075358,1311748,3000.0,3000.0,3000.0,60 months,12.69,67.79,B,B5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
# Split the dataframe into multiple dataframe based on loan status
status=['Charged Off', 'Fully Paid']
inactive_loans = loanapp_df[loanapp_df.loan_status.isin(status)]
current_loans = loanapp_df[loanapp_df.loan_status=='Current']

In [4]:
# Drop the fields with null values for more than 97%

inactive_loans.drop(['next_pymnt_d', 'annual_inc_joint', 'annual_inc_joint', 'dti_joint', 'verification_status_joint', 'open_acc_6m', 'open_il_6m', 'open_il_12m', 'open_il_24m', 'mths_since_rcnt_il', 'total_bal_il', 'il_util', 'open_rv_12m', 'open_rv_24m', 'max_bal_bc', 'all_util', 'inq_fi', 'total_cu_tl', 'inq_last_12m'], axis = 1, inplace=True)
current_loans.drop(['next_pymnt_d', 'annual_inc_joint', 'annual_inc_joint', 'dti_joint', 'verification_status_joint', 'open_acc_6m', 'open_il_6m', 'open_il_12m', 'open_il_24m', 'mths_since_rcnt_il', 'total_bal_il', 'il_util', 'open_rv_12m', 'open_rv_24m', 'max_bal_bc', 'all_util', 'inq_fi', 'total_cu_tl', 'inq_last_12m'], axis = 1, inplace=True)

/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4102: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [5]:
# Transform the field - verification_status

inactive_loans['verification_status'] = inactive_loans['verification_status'].replace(['Source Verified'],'Verified')
current_loans['verification_status'] = current_loans['verification_status'].replace(['Source Verified'],'Verified')
inactive_loans.head()

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,last_pymnt_amnt,last_credit_pull_d,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,acc_now_delinq,tot_coll_amt,tot_cur_bal,total_rev_hi_lim
0,1077501,1296599,5000.0,5000.0,4975.0,36 months,10.65,162.87,B,B2,...,171.62,Jan-2016,0.0,NaN,1.0,INDIVIDUAL,0.0,NaN,NaN,NaN
1,1077430,1314167,2500.0,2500.0,2500.0,60 months,15.27,59.83,C,C4,...,119.66,Sep-2013,0.0,NaN,1.0,INDIVIDUAL,0.0,NaN,NaN,NaN
2,1077175,1313524,2400.0,2400.0,2400.0,36 months,15.96,84.33,C,C5,...,649.91,Jan-2016,0.0,NaN,1.0,INDIVIDUAL,0.0,NaN,NaN,NaN
3,1076863,1277178,10000.0,10000.0,10000.0,36 months,13.49,339.31,C,C1,...,357.48,Jan-2015,0.0,NaN,1.0,INDIVIDUAL,0.0,NaN,NaN,NaN
5,1075269,1311441,5000.0,5000.0,5000.0,36 months,7.90,156.46,A,A4,...,161.03,Sep-2015,0.0,NaN,1.0,INDIVIDUAL,0.0,NaN,NaN,NaN


In [6]:
# Convert the categorical values into dummies in inactive loan dataframe


dummies1 = pd.get_dummies(inactive_loans[['term', 'grade', 'sub_grade', 'verification_status','purpose', 'emp_length','home_ownership']], drop_first=True)
inactive_loans.loan_status.unique()
inactive_loans = pd.concat([inactive_loans.drop(['term', 'grade', 'sub_grade', 'verification_status','purpose', 'emp_length','home_ownership'],axis=1), dummies1],axis=1)
inactive_loans.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,int_rate,installment,emp_title,annual_inc,issue_d,...,emp_length_6 years,emp_length_7 years,emp_length_8 years,emp_length_9 years,emp_length_< 1 year,home_ownership_MORTGAGE,home_ownership_NONE,home_ownership_OTHER,home_ownership_OWN,home_ownership_RENT
0,1077501,1296599,5000.0,5000.0,4975.0,10.65,162.87,NaN,24000.0,Dec-2011,...,0,0,0,0,0,0,0,0,0,1
1,1077430,1314167,2500.0,2500.0,2500.0,15.27,59.83,Ryder,30000.0,Dec-2011,...,0,0,0,0,1,0,0,0,0,1
2,1077175,1313524,2400.0,2400.0,2400.0,15.96,84.33,NaN,12252.0,Dec-2011,...,0,0,0,0,0,0,0,0,0,1
3,1076863,1277178,10000.0,10000.0,10000.0,13.49,339.31,AIR RESOURCES BOARD,49200.0,Dec-2011,...,0,0,0,0,0,0,0,0,0,1
5,1075269,1311441,5000.0,5000.0,5000.0,7.90,156.46,Veolia Transportaton,36000.0,Dec-2011,...,0,0,0,0,0,0,0,0,0,1


In [7]:
# Target variable 

status_dummy = pd.get_dummies(inactive_loans, columns=['loan_status'])
inactive_loans=status_dummy
inactive_loans.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,int_rate,installment,emp_title,annual_inc,issue_d,...,emp_length_8 years,emp_length_9 years,emp_length_< 1 year,home_ownership_MORTGAGE,home_ownership_NONE,home_ownership_OTHER,home_ownership_OWN,home_ownership_RENT,loan_status_Charged Off,loan_status_Fully Paid
0,1077501,1296599,5000.0,5000.0,4975.0,10.65,162.87,NaN,24000.0,Dec-2011,...,0,0,0,0,0,0,0,1,0,1
1,1077430,1314167,2500.0,2500.0,2500.0,15.27,59.83,Ryder,30000.0,Dec-2011,...,0,0,1,0,0,0,0,1,1,0
2,1077175,1313524,2400.0,2400.0,2400.0,15.96,84.33,NaN,12252.0,Dec-2011,...,0,0,0,0,0,0,0,1,0,1
3,1076863,1277178,10000.0,10000.0,10000.0,13.49,339.31,AIR RESOURCES BOARD,49200.0,Dec-2011,...,0,0,0,0,0,0,0,1,0,1
5,1075269,1311441,5000.0,5000.0,5000.0,7.90,156.46,Veolia Transportaton,36000.0,Dec-2011,...,0,0,0,0,0,0,0,1,0,1


In [8]:
# Scale the numeric features
from sklearn.preprocessing import StandardScaler
x=inactive_loans[['loan_amnt','funded_amnt','funded_amnt_inv','int_rate','installment','annual_inc','dti','delinq_2yrs','inq_last_6mths','mths_since_last_delinq','mths_since_last_record','open_acc','pub_rec','revol_bal','revol_util','total_acc','total_pymnt','total_pymnt_inv','total_rec_prncp','total_rec_int','total_rec_late_fee','recoveries','collection_recovery_fee','collections_12_mths_ex_med','mths_since_last_major_derog','mths_since_last_major_derog','acc_now_delinq','tot_coll_amt','tot_cur_bal','total_rev_hi_lim']]
x=x.fillna(0)
scaler=StandardScaler().fit(x)
x_scaled= scaler.transform(x)
x_scaled_df=pd.DataFrame(x_scaled, columns=x.columns)
x_scaled_df.head()

,loan_amnt,funded_amnt,funded_amnt_inv,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,mths_since_last_delinq,...,total_rec_late_fee,recoveries,collection_recovery_fee,collections_12_mths_ex_med,mths_since_last_major_derog,mths_since_last_major_derog,acc_now_delinq,tot_coll_amt,tot_cur_bal,total_rev_hi_lim
0,-1.053009,-1.050930,-1.040172,-0.707561,-1.042092,-0.825316,1.425902,-0.337449,0.138268,-0.686445,...,-0.131759,-0.212988,-0.146876,-0.075556,-0.421016,-0.421016,-0.051443,-0.008355,-0.714341,-0.777085
1,-1.360447,-1.359222,-1.344688,0.341982,-1.462779,-0.723296,-1.995730,-0.337449,3.887543,-0.686445,...,-0.131759,-0.056484,-0.137145,-0.075556,-0.421016,-0.421016,-0.051443,-0.008355,-0.714341,-0.777085
2,-1.372745,-1.371553,-1.356991,0.498732,-1.362751,-1.025073,-1.004548,-0.337449,1.075587,-0.686445,...,-0.131759,-0.212988,-0.146876,-0.075556,-0.421016,-0.421016,-0.051443,-0.008355,-0.714341,-0.777085
3,-0.438133,-0.434346,-0.421914,-0.062388,-0.321730,-0.396830,0.443708,-0.337449,0.138268,0.854194,...,2.992773,-0.212988,-0.146876,-0.075556,-0.421016,-0.421016,-0.051443,-0.008355,-0.714341,-0.777085
4,-1.053009,-1.050930,-1.037096,-1.332289,-1.068262,-0.621275,-0.686137,-0.337449,2.012906,-0.686445,...,-0.131759,-0.212988,-0.146876,-0.075556,-0.421016,-0.421016,-0.051443,-0.008355,-0.714341,-0.777085


In [10]:
# Top 7 numeric features that determine charge off using Forward Feature Selection

from mlxtend.feature_selection import SequentialFeatureSelector as sfs
from sklearn.linear_model import LogisticRegression

y=inactive_loans['loan_status_Charged Off']
X= x_scaled_df
lreg = LogisticRegression()
sfs1 = sfs(lreg, k_features=7, forward=True, verbose=2, scoring='neg_mean_squared_error')
sfs1 = sfs1.fit(X, y)
feat_names = list(sfs1.k_feature_names_)
print(feat_names)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.5s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  30 out of  30 | elapsed:   15.1s finished

[2022-05-22 12:14:47] Features: 1/7 -- score: -0.09017244179565773[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  29 out of  29 | elapsed:   36.2s finished

[2022-05-22 12:15:23] Features: 2/7 -- score: -0.07225331446641589[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.3s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  28 out of  28 | elapsed:   39.0s finished

[2022-05-22 12:16:02] Features: 3/7 -- score: -0.01001299259920126[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Don

['funded_amnt', 'delinq_2yrs', 'total_pymnt', 'total_rec_prncp', 'total_rec_int', 'recoveries', 'mths_since_last_major_derog']


[Parallel(n_jobs=1)]: Done  24 out of  24 | elapsed:   52.8s finished

[2022-05-22 12:19:22] Features: 7/7 -- score: -0.009186814239091788

In [11]:

# Select the top 10 features using chi2

from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2, f_classif

x= inactive_loans[['term_ 60 months','grade_B','grade_C','grade_D','grade_E', 'grade_F', 'grade_G', 'verification_status_Verified','emp_length_10+ years','emp_length_3 years', 'emp_length_4 years', 'emp_length_2 years','emp_length_5 years', 'emp_length_6 years','emp_length_7 years', 'emp_length_8 years','emp_length_9 years','emp_length_< 1 year','home_ownership_MORTGAGE', 'home_ownership_NONE', 'home_ownership_OTHER', 'home_ownership_OWN', 'home_ownership_RENT']]
chi2_selector= SelectKBest(chi2, k=10)
features_kbest=chi2_selector.fit_transform(x,y)

print("Original number of features:", x.shape[1])
print("Reduced number of features:", features_kbest.shape[1])

# Obtain the feature names of the top 5 influencing features
cols = chi2_selector.get_support(indices=True)
features_df_new = x.iloc[:,cols]
features_df_new.head()

Original number of features: 23
Reduced number of features: 10


,term_ 60 months,grade_B,grade_C,grade_D,grade_E,grade_F,grade_G,verification_status_Verified,home_ownership_MORTGAGE,home_ownership_RENT
0,0,1,0,0,0,0,0,1,0,1
1,1,0,1,0,0,0,0,1,0,1
2,0,0,1,0,0,0,0,0,0,1
3,0,0,1,0,0,0,0,1,0,1
5,0,0,0,0,0,0,0,1,0,1


In [63]:
# Selected features
# Train - Test split

from sklearn.model_selection import train_test_split

X=inactive_loans[['term_ 60 months','grade_B', 'grade_C', 'grade_D','grade_E', 'grade_F', 'grade_G', 'verification_status_Verified', 'home_ownership_MORTGAGE','home_ownership_RENT','funded_amnt', 'delinq_2yrs', 'total_pymnt', 'total_rec_prncp', 'total_rec_int', 'recoveries']]
y=inactive_loans[['loan_status_Charged Off']]
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20)
X.dropna()
y.dropna()
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

(202376, 23) (50595, 23) (202376, 1) (50595, 1)


In [64]:
# model selection

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline


pipe=Pipeline([("classifier", RandomForestClassifier())])

search_space=[{"classifier": [LogisticRegression()], "classifier__penalty":['l2'], "classifier__C": np.logspace(0,4,10)}, {"classifier": [RandomForestClassifier()], "classifier__n_estimators":[10,100,1000], "classifier__max_features":[1,2,3]}]

In [66]:
# Grid search to obtain the best model

clf = GridSearchCV(pipe, search_space, cv=5, verbose=0, n_jobs = -1)
best_model = clf.fit(x_train, y_train)
print(best_model.best_estimator_.get_params()["classifier"])

/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression()


/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


In [67]:
# Best Model Score

clf.score(x_test, y_test)

0.8222156339559245

                                Conclusion
                                
 GridSearchCV passed the training dataset through both logistic regression and random forest models and determined that logistc regression is the best model for determining if a loan will likely be charged off. The logistic regression model also has the best model score of 82.22%